In [28]:
import threading
import multiprocessing
import queue
from tqdm import tqdm_notebook
import time

class CheckerThread(threading.Thread):
    def __init__(self, q_done, expc, th_list, name=None):
        super(CheckerThread, self).__init__()
        self.name = name
        self.kill_me = False
        self.th_list = th_list
        self.q_done = q_done
        self.expc = expc
        return

    def run(self):
        last = 0
        with tqdm_notebook(total=self.expc) as tq:
            while self.q_done.qsize() < self.expc and self.kill_me is False:
                new = self.q_done.qsize()
                tq.update(new - last)
                time.sleep(1)
                last = new
            if self.q_done.qsize() == self.expc:
                tq.update(self.expc - last)
        for t in self.th_list:
            t.stop()
        return

    def stop(self):
        for t in self.th_list:
            t.stop()
        self.kill_me = True


class ConsumerProducerThread(threading.Thread):
    def __init__(self, q_todo, q_done, expc, function, name=None):
        super(ConsumerProducerThread, self).__init__()
        self.name = name
        self.kill_me = False
        self.q_todo = q_todo
        self.q_done = q_done
        self.function = function
        self.expc = expc
        return
        
    def logic(self):
        if not self.q_todo.empty():
            item = self.q_todo.get()
            is_ok, output = self.function(item)
            if not is_ok:
                self.q_todo.put(item)
            else:
                self.q_done.put([item, output])

    def run(self):
        while self.q_done.qsize() < self.expc and self.kill_me is False:
            self.logic()
        return

    def stop(self):
        self.kill_me = True


def multiqueue(input_list, function, n_cores=None, expc=None):
    if n_cores is None:
        cores = multiprocessing.cpu_count()
        n_cores = max(1, cores-1)

    if expc is None:
        expc = len(input_list)
    
    q_todo = queue.Queue()
    q_done = queue.Queue()
    for i in input_list:
        q_todo.put(i) 
    
    th = [ConsumerProducerThread(q_todo, q_done, expc, function, name='th' + str(i)) for i in range(n_cores)]
    for t in th:
        t.start()

    ch = CheckerThread(q_done, expc, th, name="th" + str(n_cores))
    ch.start()
    ch.join()

    return list(q_done.queue)

In [38]:
def a_test(x):
    time.sleep(5)
    return True, x

res = multiqueue(list(range(10)), function=a_test)

In [39]:
res = [a_test(x) for x in tqdm_notebook(range(10))]

In [34]:
res

[(True, 10),
 (True, 11),
 (True, 12),
 (True, 13),
 (True, 14),
 (True, 15),
 (True, 16),
 (True, 17),
 (True, 18),
 (True, 19),
 (True, 20),
 (True, 21),
 (True, 22),
 (True, 23),
 (True, 24),
 (True, 25),
 (True, 26),
 (True, 27),
 (True, 28),
 (True, 29),
 (True, 30),
 (True, 31),
 (True, 32),
 (True, 33),
 (True, 34),
 (True, 35),
 (True, 36),
 (True, 37),
 (True, 38),
 (True, 39),
 (True, 40),
 (True, 41),
 (True, 42),
 (True, 43),
 (True, 44),
 (True, 45),
 (True, 46),
 (True, 47),
 (True, 48),
 (True, 49),
 (True, 50),
 (True, 51),
 (True, 52),
 (True, 53),
 (True, 54),
 (True, 55),
 (True, 56),
 (True, 57),
 (True, 58),
 (True, 59),
 (True, 60),
 (True, 61),
 (True, 62),
 (True, 63),
 (True, 64),
 (True, 65),
 (True, 66),
 (True, 67),
 (True, 68),
 (True, 69),
 (True, 70),
 (True, 71),
 (True, 72),
 (True, 73),
 (True, 74),
 (True, 75),
 (True, 76),
 (True, 77),
 (True, 78),
 (True, 79),
 (True, 80),
 (True, 81),
 (True, 82),
 (True, 83),
 (True, 84),
 (True, 85),
 (True, 86),